In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Cropping2D
import os
import matplotlib.image as mpimg
import cv2

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9556034116768367092
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 7640199988
locality {
  bus_id: 1
}
incarnation: 2140360935880139020
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0"
]


In [3]:
pwd

'/notebooks/USDCN-project3_Behavioural_Cloning'

In [4]:
old_path = '/home/sergio/workspace/sim_data/'

In [5]:
def fix_path(data_df):
    cols = data_df.columns.tolist()
    for col in data_df.columns:
        try:
            data_df[col] = data_df[col].apply(lambda x: x.replace(old_path, ''))
        except:
            pass
    return data_df

In [6]:
def augment_brightness(image):
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    image1 = np.array(image1, dtype = np.float64)
    random_bright = .5+np.random.uniform()
    image1[:,:,2] = image1[:,:,2]*random_bright
    image1[:,:,2][image1[:,:,2]>255]  = 255
    image1 = np.array(image1, dtype = np.uint8)
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

def add_shadow(image):
    top_y = 320*np.random.uniform()
    top_x = 0
    bot_x = 160
    bot_y = 320*np.random.uniform()
    image_hls = cv2.cvtColor(image,cv2.COLOR_RGB2HLS)
    shadow_mask = 0*image_hls[:,:,1]
    X_m = np.mgrid[0:image.shape[0],0:image.shape[1]][0]
    Y_m = np.mgrid[0:image.shape[0],0:image.shape[1]][1]

    shadow_mask[((X_m-top_x)*(bot_y-top_y) -(bot_x - top_x)*(Y_m-top_y) >=0)]=1
    #random_bright = .25+.7*np.random.uniform()
    if np.random.randint(2)==1:
        random_bright = .5
        cond1 = shadow_mask==1
        cond0 = shadow_mask==0
        if np.random.randint(2)==1:
            image_hls[:,:,1][cond1] = image_hls[:,:,1][cond1]*random_bright
        else:
            image_hls[:,:,1][cond0] = image_hls[:,:,1][cond0]*random_bright    
    image = cv2.cvtColor(image_hls,cv2.COLOR_HLS2RGB)

    return image

def augment_images(data_dir, center, left, right, steering_angle):
    # Choose an image from left, center or right and adjust steering angle
    choice = np.random.choice(3)
    if choice == 0:
        image = mpimg.imread(os.path.join(data_dir, left.strip()))
        steering_angle += 0.2
    elif choice == 1:
        image = mpimg.imread(os.path.join(data_dir, right.strip()))
        steering_angle -= 0.2
    elif choice ==2:
        image = mpimg.imread(os.path.join(data_dir, center.strip()))

    # make a random flip on the image
    if np.random.rand() < 0.5:
        image = cv2.flip(image, 1)
        steering_angle = -steering_angle

    return image, steering_angle

In [7]:
def batch_generator(data_dir, image_paths, steering_angles, batch_size, is_training):
    # Generate training image 
    images = np.empty([batch_size, IM_HEIGHT, IM_WIDTH, IM_CHANNELS])
    steers = np.empty(batch_size)
    while True:
        i = 0
        for index in np.random.permutation(image_paths.shape[0]):
            center, left, right = image_paths[index]
            steering_angle = steering_angles[index]
            if is_training and np.random.rand() < 0.6:
                # augment data when in training
                image, steering_angle = augment_images(data_dir, center, left, right, steering_angle)
                
                
            else:
                # chooses image from center
                image = mpimg.imread(os.path.join(data_dir, center.strip()))
            
            
            # add image and steering angle
            images[i] = add_shadow(augment_brightness(image))
            steers[i] = steering_angle
            i += 1
            if i == batch_size:
                break
        yield images, steers

In [8]:
np.random.seed(42)
IM_HEIGHT = 160
IM_WIDTH = 320
IM_CHANNELS = 3

# load data
data_dir = './data2/'
test_size = .25

data_df = pd.read_csv(os.path.join(data_dir,'driving_log_final.csv'))
data_df.columns = ['center','left','right','steering','throttle','break','speed']

data_df = fix_path(data_df)

X = data_df[['center', 'left', 'right']].values
y = data_df['steering'].values

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=42)

In [9]:
data_df.head()


,center,left,right,steering,throttle,break,speed
0,IMG/center_2016_12_01_13_32_43_558.jpg,IMG/left_2016_12_01_13_32_43_558.jpg,IMG/right_2016_12_01_13_32_43_558.jpg,0.052191,0.985533,0.0,3.286475
1,IMG/center_2016_12_01_13_32_43_659.jpg,IMG/left_2016_12_01_13_32_43_659.jpg,IMG/right_2016_12_01_13_32_43_659.jpg,0.052191,0.985533,0.0,4.440864
2,IMG/center_2016_12_01_13_32_43_761.jpg,IMG/left_2016_12_01_13_32_43_761.jpg,IMG/right_2016_12_01_13_32_43_761.jpg,0.367953,0.985533,0.0,5.565724
3,IMG/center_2016_12_01_13_32_43_862.jpg,IMG/left_2016_12_01_13_32_43_862.jpg,IMG/right_2016_12_01_13_32_43_862.jpg,0.578461,0.985533,0.0,6.626935
4,IMG/center_2016_12_01_13_32_43_963.jpg,IMG/left_2016_12_01_13_32_43_963.jpg,IMG/right_2016_12_01_13_32_43_963.jpg,0.578461,0.985533,0.0,7.730138


In [10]:
print(len(X_train), len(X_valid))
print(len(X_train) + len(X_valid))

5443 1815
7258


In [11]:
# build keras model
INPUT_SHAPE = (IM_HEIGHT, IM_WIDTH, IM_CHANNELS)
keep_prob = .55

model = Sequential()
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))
model.add(Lambda(lambda x: x/127.5-1.0))

model.add(Conv2D(24, 5, activation='elu', strides=(2, 2)))
model.add(Conv2D(36, 5, activation='elu', strides=(2, 2)))
model.add(Conv2D(48, 5, activation='elu', strides=(2, 2)))
model.add(Conv2D(64, 3, activation='elu'))
model.add(Conv2D(64, 3, activation='elu'))

model.add(Dropout(keep_prob))

model.add(Flatten())

model.add(Dense(100, activation='elu'))
model.add(Dense(50, activation='elu'))
model.add(Dense(10, activation='elu'))
model.add(Dense(1))

model.summary()

# train model
learning_rate = 1.0e-4
batch_size = 64
steps_per_epoch = 500
nb_epoch = 10

model.compile(loss='mean_squared_error', optimizer=Adam(lr=learning_rate))

model.fit_generator(batch_generator(data_dir, X_train, y_train, batch_size, True),
                    steps_per_epoch,
                    nb_epoch,
                    max_queue_size=1,
                    validation_data=batch_generator(data_dir, X_valid, y_valid, batch_size, False),
                    validation_steps=len(X_valid),
                    verbose=1) 
print('saving')
model.save('model.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cropping2d_1 (Cropping2D)    (None, 90, 320, 3)        0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 90, 320, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 43, 158, 24)       1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 77, 36)        21636     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 37, 48)         43248     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 6, 35, 64)         27712     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4, 33, 64)         36928     
__________